## Determine main melody 
 - by score (author specified)
 - pan (左右聲道平衡度)
 - velociy (按鍵力道)
 - 平均旋律
 - 和弦所佔比率

In [32]:
main_name = ['melody','melodies','vocal','voice','sing','lead']
sub_name = ['drum']

## pan value

In [33]:
def panDect(parts):
    pan = {}
    [pan.update({part.Staff.get('id'):part.select('Channel')[0].controller.get('value')}) for part in parts] # balance value: 63
    return pan

## average velocity

In [34]:
def velDect(addr,parts):
    sta_id = [part.Staff.get('id') for part in parts]
    fm = open(addr, 'r')
    music = fm.read()
    fm.close()
    vela = {}
    staffs = [staff for staff in bs(music,'xml').select('Score > Staff') if staff.get('id') in sta_id]
    for staff in staffs:
        vels = staff.select('Note velocity')
        n = len(vels)
        s = 0.0
        for vel in vels:
            v = int(vel.text)
            s += v
        avg = s/n
        vela.update({staff.get('id'):str(avg)})
    return vela

## average pitch

In [35]:
import math
def pitDect(addr,parts):
    sta_id = [part.Staff.get('id') for part in parts]
    fm = open(addr, 'r')
    music = fm.read()
    fm.close()
    pits = {}
    dutype = {'whole':1.0,'half':1.0/2,'quarter':1.0/4,'eighth':1.0/8,'16th':1.0/16,'32nd':1.0/32,'64th':1.0/64,'128th':1.0/128}
    staffs = [staff for staff in bs(music,'xml').select('Score > Staff') if staff.get('id') in sta_id]
    for staff in staffs:
        pitch_sum = 0
        pitch_time = 0
        for measure in staff.select('Measure'):
            for chord in [c for c in list(measure.children) if c != '\n' and c.name == 'Chord']:
                max_pitch = max([int(d.pitch.text) for d in chord.select('Note')])
                dot = 0
                if len(chord.select('dot')) != 0:
                    dot = int(chord.select('dot')[0].text)
                dutime = dutype[chord.durationType.text] * (2-math.pow(1.0/2,dot))
                pitch_sum += max_pitch * dutime
                pitch_time += dutime
        avg_pich = pitch_sum/pitch_time
        pits.update({staff.get('id'):str(avg_pich)})
    return pits

## chords' rate

In [36]:
def rateDect(addr,parts):
    sta_id = [part.Staff.get('id') for part in parts]
    fm = open(addr, 'r')
    music = fm.read()
    fm.close()
    rates = {}
    staffs = [staff for staff in bs(music,'xml').select('Score > Staff') if staff.get('id') in sta_id]
    for staff in staffs:
        num = 0
        n = 0
        for measure in staff.select('Measure'):
            chords = [c for c in list(measure.children) if c != '\n' and c.name == 'Chord']
            n += len(chords)
            for chord in chords:
                if len(chord.select('Note')) == 1:
                    num += 1
        rates.update({staff.get('id'):str(float(num)/n)})
    return rates

## sound area

In [52]:
def areaDect(addr,parts):
    sta_id = [part.Staff.get('id') for part in parts]
    fm = open(addr, 'r')
    music = fm.read()
    fm.close()
    area = {}
    staffs = [staff for staff in bs(music,'xml').select('Score > Staff') if staff.get('id') in sta_id]
    dutype = {'whole':1.0,'half':1.0/2,'quarter':1.0/4,'eighth':1.0/8,'16th':1.0/16,'32nd':1.0/32,'64th':1.0/64,'128th':1.0/128}
    for staff in staffs:
        du = 0
        all_du = []
        for measure in staff.select('Measure'):
            chords = [c for c in list(measure.children) if c != '\n' and c.name == 'Chord']
            for chord in chords:
                all_du.append(dutype[chord.durationType.text])
        area.update({staff.get('id'):str(sum(all_du))})
    return area

## determine main melodies

In [43]:
import pandas as pd
def detMain(data):
    form = pd.DataFrame(data).transpose()
    form.columns = ['pan','vel_rate','pit_rate','cho_rate','area']
    # need to add...
    return form       

## main function

In [61]:
from bs4 import BeautifulSoup as bs
addr = 'C:/Users/BigData/Desktop/datas/python/py/Holding Out for a Hero.mscx'
fm = open(addr, 'r')
music = fm.read()
fm.close()
cand = bs(music,'xml').select('Part')
main_cand = []
sub_cand = []
for s in [par for par in cand if len(par.select('longName')) != 0]:
    pname = s.select('longName')[0].text.encode('utf8').lower()
#     if len([word for word in main_name if word in pname]) != 0: # instru. name
#         main_cand.append(s)
#         cand.remove(s)
    if len([word for word in sub_name if word in pname]) != 0:
        sub_cand.append(s)
        cand.remove(s)
    for cin in [c for c in list(s.select('Instrument')[0].children) if c != '\n']: # more than one instru. in this staff
        if cin.get('pitch') is not None:
            if s in cand:
                sub_cand.append(s)
                cand.remove(s)
            break
mesg = {}
pan = panDect(cand)
vels = velDect(addr,cand)
pits = pitDect(addr,cand)
rates = rateDect(addr,cand)
area = areaDect(addr,cand)
[mesg.update({a:[pan[a],vels[a],pits[a],rates[a],area[a]]}) for a in pan.keys()]
f = detMain(mesg)
print f

  pan       vel_rate       pit_rate        cho_rate      area
1  63  122.644189383  67.9914040115             1.0  109.0625
2  63  123.910404624  67.6733232856  0.262845849802   165.875
3  63  123.710655738  46.2376943005             1.0     193.0
4  63  102.720491803  46.2376943005             1.0     193.0
5  63  106.305785124  71.3319672131            0.75      30.5
6  63          125.0  57.6213592233  0.341176470588    12.875


[]


   0  1  2
1  a  b  c
2  a  c  g


[1, [2, 3.5, 5]]
